# nginx-gzip压缩提升网站性能

nginx可以把静态文件压缩好之后再传给浏览器。只要浏览器的请求头带上Accept-Encoding: gzip就可以了。假如有一个文件叫application.css，那nginx就会使用gzip模块把这个文件压缩，然后传给浏览器，浏览器再解压缩成原来的css文件，就能读取了。

需要nginx的ngx_http_gzip_module模块。这个模块能对需要的静态文件压缩大小，比如图片，css，javascript，html等。压缩是需要消耗CPU，但能提高传缩的速度，因为传缩量少了许多，从而节省带宽。

### 使用gzip压缩文件

1. 查看nginx是否编译了ngx_http_gzip_module这个模块: `sudo nginx -V`, 如果输出`--with-ngx_http_gzip_module`，说明已经编译了。没有的话，就需要编译了.

2. 配置gzip

```
http {
        gzip on;
        gzip_disable "msie6";

        gzip_vary on;
        gzip_proxied any;
        gzip_comp_level 6;
        gzip_buffers 16 8k;
        gzip_http_version 1.1;
        gzip_types text/plain text/css application/json application/x-javascript text/xml application/xml application/xml+rss text/javascript;
        server {
                location ~ ^/assets/ {
                   gzip_static on;
                   expires max;
                   add_header Cache-Control public;
                }        
        }
}
```

上面最重要的是http中`gzip on` 和 `gzip_types`这两行，是一定要写的。其他的`gzip_vary`等都是一些配置，可以不写。

3. 在需要压缩的静态资源那里加上下面三行

```
gzip_static on;
expires max;
add_header Cache-Control public;
```

4. 如果改了配置用`sudo nginx -s reload`重新加载生效。

5. 测试

    1. 使用浏览器，比如chrome。只要用开发者的network功能查看两次资源的大小就好了。

    2. 使用curl工具也可以。例如: `curl -I -H "Accept-Encoding: gzip" http://www.rails365.net/assets/application-7166fb1597da14300423c6a60c281a45f24573b249eafe0fd84b5c261db1d3a5.js`, 只要返回Content-Encoding: gzip说明成功的